<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #11: Twitter
`Fecha de entrega: Mayo 20, 2021 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [1]:
import pandas as pd
import tweepy

In [2]:
# Leer las llaves
keys = pd.read_csv("twitter.csv", header=None)
keys = dict(zip(keys[0],keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [3]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
# Paso 4
tweets = tweepy.Cursor(api.search,
                       q=['S.O.S colombia -RT'],
                       lang='es',
                       geocode='4.7110,-74.0721,100km',
                       tweet_mode='extended',
                       include_rts=False,
                       ).items(100)

In [5]:
# Objeto generador
t = [tweet for tweet in tweets]

In [6]:
t_processed = []

for tweet in t: 
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append('Bogotá')
    temp.append(tweet.user.location)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/stuatus/{tweet.id}")
    
    t_processed.append(temp)
    
    
data = pd.DataFrame(t_processed)
data.columns = ['tweets', 'date', 'city', 'user_location', 'id', 'link']
data.head()


,tweets,date,city,user_location,id,link
0,@CNNEE Auxilio se enloqueció la policia en var...,2021-05-20 04:22:25,Bogotá,"Bogotá, D.C., Colombia",1395233425037176834,https://twitter.com/i/web/stuatus/139523342503...
1,"😡😡😡 @DirectorPolicia Sigan guardando silencio,...",2021-05-20 03:26:17,Bogotá,"BOGOTA, COLOMBIA",1395219300517126147,https://twitter.com/i/web/stuatus/139521930051...
2,S&amp;P bajó la calificación de la deuda sober...,2021-05-20 02:51:37,Bogotá,Bogotá,1395210575324532742,https://twitter.com/i/web/stuatus/139521057532...
3,S.O.S Colombia: golpe de Estado https://t.co/b...,2021-05-20 02:25:57,Bogotá,"Pereira, Colombia",1395204115022684164,https://twitter.com/i/web/stuatus/139520411502...
4,¿Ya salió Vicky Dávila a decir que Standard an...,2021-05-20 02:00:15,Bogotá,"Bogotá, D.C., Colombia",1395197646978093059,https://twitter.com/i/web/stuatus/139519764697...


In [8]:
data = data[['tweets', 'date']]
data.head()

,tweets,date
0,@CNNEE Auxilio se enloqueció la policia en var...,2021-05-20 04:22:25
1,"😡😡😡 @DirectorPolicia Sigan guardando silencio,...",2021-05-20 03:26:17
2,S&amp;P bajó la calificación de la deuda sober...,2021-05-20 02:51:37
3,S.O.S Colombia: golpe de Estado https://t.co/b...,2021-05-20 02:25:57
4,¿Ya salió Vicky Dávila a decir que Standard an...,2021-05-20 02:00:15


In [9]:
import re 

pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets']
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions

In [10]:
  
data[['texto_limpio', 'links', 'hashtags', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')
data

,tweets,date,texto_limpio,links,hashtags,mentions
0,@CNNEE Auxilio se enloqueció la policia en var...,2021-05-20 04:22:25,Auxilio se enloqueció la policia en varias pa...,[],[],[@CNNEE]
1,"😡😡😡 @DirectorPolicia Sigan guardando silencio,...",2021-05-20 03:26:17,"😡😡😡 Sigan guardando silencio, que nosotros Se...",[https://t.co/vCHfdXX7zp],[],"[@DirectorPolicia, @ONUHumanRights, @CorteIDH,..."
2,S&amp;P bajó la calificación de la deuda sober...,2021-05-20 02:51:37,S&amp;P bajó la calificación de la deuda sober...,[],[],[]
3,S.O.S Colombia: golpe de Estado https://t.co/b...,2021-05-20 02:25:57,S.O.S Colombia: golpe de Estado,[https://t.co/bOIOcVQYyr],[],[]
4,¿Ya salió Vicky Dávila a decir que Standard an...,2021-05-20 02:00:15,¿Ya salió Vicky Dávila a decir que Standard an...,[],[],[]
...,...,...,...,...,...,...
95,@VarskySports Por favor s.o.s nos están matand...,2021-05-14 02:53:07,Por favor s.o.s nos están matando en Colombia 🥺,[],[],[@VarskySports]
96,Organizaciones de derechos humanos en el mundo...,2021-05-14 00:47:29,Organizaciones de derechos humanos en el mundo...,[https://t.co/TyGX5QgajI],[#SOSColombiaDDHH],[@ChangeorgCO]
97,S.O.S. COLOMBIA RESISTE A LAS MASACRES DE ESTE...,2021-05-13 23:08:13,S.O.S. COLOMBIA RESISTE A LAS MASACRES DE ESTE...,[https://t.co/cPDF2FhtJN],[],[]
98,Disponible en 👇\nhttps://t.co/paO1YrJnr6\n+ In...,2021-05-13 23:00:09,Disponible en 👇\n\n+ Inf 📲 3115928048. ...,"[https://t.co/paO1YrJnr6, https://t.co/bNQkqrb...","[#colombia, #cantantes, #musica, #music, #cant...",[]


In [11]:
import regex
import emoji

def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)

    return emoji_list
  
  
data['emojis'] = data['texto_limpio'].apply(lambda text: get_emojis(text))
data.head()


,tweets,date,texto_limpio,links,hashtags,mentions,emojis
0,@CNNEE Auxilio se enloqueció la policia en var...,2021-05-20 04:22:25,Auxilio se enloqueció la policia en varias pa...,[],[],[@CNNEE],[]
1,"😡😡😡 @DirectorPolicia Sigan guardando silencio,...",2021-05-20 03:26:17,"😡😡😡 Sigan guardando silencio, que nosotros Se...",[https://t.co/vCHfdXX7zp],[],"[@DirectorPolicia, @ONUHumanRights, @CorteIDH,...","[😡, 😡, 😡]"
2,S&amp;P bajó la calificación de la deuda sober...,2021-05-20 02:51:37,S&amp;P bajó la calificación de la deuda sober...,[],[],[],[]
3,S.O.S Colombia: golpe de Estado https://t.co/b...,2021-05-20 02:25:57,S.O.S Colombia: golpe de Estado,[https://t.co/bOIOcVQYyr],[],[],[]
4,¿Ya salió Vicky Dávila a decir que Standard an...,2021-05-20 02:00:15,¿Ya salió Vicky Dávila a decir que Standard an...,[],[],[],[]


In [12]:
data['hora'] = data['date'].dt.floor('T').dt.time
temp = pd.DataFrame(data.hora.value_counts()).reset_index()
temp.columns = ['hora', 'cnt']
temp = temp.sort_values('hora')
temp

,hora,cnt
60,00:10:00,1
65,00:19:00,1
47,00:47:00,1
71,01:06:00,1
78,01:12:00,1
...,...,...
64,23:08:00,1
70,23:12:00,1
38,23:24:00,1
21,23:36:00,1


In [13]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [14]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tuits por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)